In [12]:
import pandas as pd 
geelong_data=pd.read_csv('/content/geelong_data.csv')
builtenv_data=pd.read_csv('/content/geelongbuiltenv.csv')
builtenv_data


,Suburb,LUM,LUMquintile,TrainDistThresh,TrainPres,BusDistThresh,BusPres,AccUtDestPres,AccUtDestPresquintile,AccUtDestDist,AccUtDestDistquintile,SprtLeiDens,SprtLeiDensquintile,SprtLeiDist,SprtLeiDistquintile,POSDist,POSDistquintile,POSAreaPct,POSAreaPctquintile,TakeawayDens,TakeawayDensquintile,TakeawayDist,TakeawayDistquintile,HealthyFoodDensity,HealthyFoodDensityquintile,HealthyFoodDist,HealthyFoodDistquintile,GrossDwlDens,GrossDwlDensquintile,StrtConn,StrtConnquintile,ActCenDist,ActCenDistquintile,TobPopDensp1000,TobPopDensp1000quintile,TobTobVendDist,TobTobVendDistquintile,RatioTrailsToRoads,RatioTrailsToRoadsquintile
0,Clifton Springs,0.418636,4,0,0,1,1,2,2,2,2,0.000000,1,2334.840293,3,195.308128,1,0.211762,4,0.088528,3,734.854534,2,0.000000,1,2681.513753,4,6.138845,4,41.378732,4,21768.170070,4,0.419933,2,775.371825,3,0.063443,2
1,Geelong West,0.512726,4,1,0,1,1,7,5,7,5,0.000000,1,913.552249,1,250.556874,3,0.031718,1,0.485292,5,341.128302,1,0.178792,5,584.101684,1,13.940743,5,77.755440,5,1812.155338,1,2.996704,4,315.721995,1,0.025997,1
2,Newcomb/Moolap,0.209689,2,0,0,1,1,5,3,4,4,0.020258,3,1400.551320,2,324.854903,4,0.425467,5,0.162064,4,769.448651,3,0.050645,2,1366.097985,2,0.676011,3,7.134546,3,5267.718643,2,2.670940,4,746.243079,2,0.135687,3
3,City/Drum/Sth Geelong,0.847546,5,1,1,1,1,7,5,7,5,0.037318,4,965.344308,1,229.975343,2,0.142600,4,0.621958,5,369.506218,1,0.024878,2,956.274668,2,7.481207,5,113.265985,5,1663.495404,1,9.479434,5,316.179185,1,0.172038,3
4,Hamlyn Heights,0.307938,3,0,0,1,1,3,2,4,3,0.000000,1,1967.155970,3,295.410346,3,0.103317,4,0.056174,3,721.547118,2,0.028087,2,932.819631,2,9.638756,5,57.587787,5,4325.191244,1,0.329327,2,669.341899,2,0.157551,3
5,Bell Park,0.543160,5,1,0,1,1,4,2,6,3,0.032629,3,1121.616791,1,192.280114,1,0.066116,1,0.261031,5,742.745672,2,0.130515,5,612.031513,1,6.863578,5,48.632306,4,5604.934726,2,1.513432,3,613.834398,2,0.284067,5
6,St Albans Park,0.133935,2,0,0,1,1,2,2,2,2,0.000000,1,1948.108506,3,242.314703,2,0.049041,1,0.039779,2,844.938885,3,0.000000,1,2430.582007,3,9.828608,5,54.203644,5,6505.646796,2,0.202716,2,839.850141,3,0.031002,1
7,Bell Post Hill,0.417682,4,0,0,1,1,4,2,4,3,0.000000,1,1546.179119,2,248.796797,3,0.049179,1,0.048314,2,697.342190,2,0.024157,2,706.692301,1,3.559913,3,19.402758,3,6247.545090,2,0.199521,2,844.401172,3,0.189429,4
8,Waurn Ponds,0.245583,3,0,0,0,1,3,2,0,3,0.014135,2,2137.784320,3,527.550215,4,0.005661,1,0.042404,2,1822.271536,4,0.000000,1,3016.184906,4,0.324108,2,5.002141,2,7143.010736,3,3.076923,5,1831.748986,4,0.145049,3
9,Grovedale/Marshall,0.533334,5,1,1,1,1,8,5,6,5,0.019016,3,2094.272727,3,277.667748,3,0.054322,1,0.133113,4,846.380240,3,0.066557,5,961.271548,2,3.168404,3,23.080450,3,7280.750816,3,0.877358,3,792.878963,3,0.166972,3


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [13]:
import findspark
findspark.init()
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
geelong_data_spark_df = spark.createDataFrame(geelong_data)
builtenv_data_spark_df = spark.createDataFrame(builtenv_data)
geelong_data_spark_df.show()
builtenv_data_spark_df.show()

+-----+--------------------+--------------------+----------+--------+--------+--------------------+--------------------+--------------------+----------+----------+----------+-----------+--------+--------+--------+
|WebID|                Area|Total_dwellings_2016|Train_2016|pop_2016|emp_2016|Total dwellings_2011|Total dwellings_2006|Total dwellings_2001|Train_2011|Train_2006|Train_2001|Unnamed: 12|pop_2011|pop_2006|pop_2001|
+-----+--------------------+--------------------+----------+--------+--------+--------------------+--------------------+--------------------+----------+----------+----------+-----------+--------+--------+--------+
| 10.0|City of Greater G...|              106149|      3864|  233429|   59000|              4075.0|              3318.0|              2839.0|      75.0|      80.0|      66.0|        NaN| 10117.0|  8818.0|  7654.0|
|100.0|Armstrong Creek -...|                1973|       116|    5432|    1891|              1326.0|              1309.0|              1279.0|   

In [14]:
geelong_data_spark_df.createOrReplaceTempView("geelong_table")
builtenv_data_spark_df.createOrReplaceTempView("builtenv_table")
co=spark.sql("select count(*) from builtenv_table")
co.show()
same_suburbs=spark.sql("select g.Area,b.Suburb from geelong_table g,builtenv_table b where g.Area=b.Suburb")
same_suburbs.show()
distinct_names=spark.sql("select k.Suburb from builtenv_table k where k.Suburb NOT IN(select b.Suburb from geelong_table g,builtenv_table b where g.Area=b.Suburb) order by k.Suburb")
distinct_names.show(100)
geelong_areas=spark.sql("select area from geelong_table order by area")
geelong_areas.show(100)


+--------+
|count(1)|
+--------+
|      46|
+--------+

+--------------------+--------------------+
|                Area|              Suburb|
+--------------------+--------------------+
|            Balliang|            Balliang|
|             Highton|             Highton|
|  Thomson/Breakwater|  Thomson/Breakwater|
|              Anakie|              Anakie|
|        Mount Duneed|        Mount Duneed|
|     Wandana Heights|     Wandana Heights|
|        Barwon Heads|        Barwon Heads|
|City/Drum/Sth Gee...|City/Drum/Sth Gee...|
|            Drysdale|            Drysdale|
|     Clifton Springs|     Clifton Springs|
|      St Albans Park|      St Albans Park|
|     Curlewis (Vic.)|     Curlewis (Vic.)|
|      Hamlyn Heights|      Hamlyn Heights|
|         Marcus Hill|         Marcus Hill|
|               Ceres|               Ceres|
|          Herne Hill|          Herne Hill|
|    Manifold Heights|    Manifold Heights|
|            Breamlea|            Breamlea|
|           Mooraboo

In [0]:
#distinct_names_pd=distinct_names.toPandas()
#from google.colab import files

#distinct_names_pd.to_csv('distinct_names.csv') 
#files.download('distinct_names.csv')

In [28]:
processed_data=spark.sql("select g.Area,g.Total_dwellings_2016,g.Train_2016,g.pop_2016,b.* from geelong_table g,builtenv_table b where g.Area=b.Suburb")
processed_data.show()
pre_processed_data=processed_data.toPandas()
pre_processed_data=pre_processed_data.drop(['Suburb'], axis = 1)
pre_processed_data
pre_processed_data.to_csv('pre_processed_data.csv') 
files.download('pre_processed_data.csv')


+--------------------+--------------------+----------+--------+--------------------+-------------------+-----------+---------------+---------+-------------+-------+-------------+---------------------+-------------+---------------------+-----------+-------------------+------------------+-------------------+------------------+---------------+--------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------------+------------------+-----------------------+-------------------+--------------------+------------------+----------------+------------------+------------------+------------------+-----------------------+------------------+----------------------+--------------------+--------------------------+
|                Area|Total_dwellings_2016|Train_2016|pop_2016|              Suburb|                LUM|LUMquintile|TrainDistThresh|TrainPres|BusDistThresh|BusPres|AccUtDestPres|AccUtDestPresquint

In [30]:
#label encoding for area and suburb
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
pre_processed_data['Area'] = labelencoder.fit_transform(pre_processed_data['Area'])
#pre_processed_data['Suburb'] = labelencoder.fit_transform(pre_processed_data['Suburb'])
#pre_processed_data=pre_processed_data.drop(['Suburb'], axis = 1)
pre_processed_data

,Area,Total_dwellings_2016,Train_2016,pop_2016,LUM,LUMquintile,TrainDistThresh,TrainPres,BusDistThresh,BusPres,AccUtDestPres,AccUtDestPresquintile,AccUtDestDist,AccUtDestDistquintile,SprtLeiDens,SprtLeiDensquintile,SprtLeiDist,SprtLeiDistquintile,POSDist,POSDistquintile,POSAreaPct,POSAreaPctquintile,TakeawayDens,TakeawayDensquintile,TakeawayDist,TakeawayDistquintile,HealthyFoodDensity,HealthyFoodDensityquintile,HealthyFoodDist,HealthyFoodDistquintile,GrossDwlDens,GrossDwlDensquintile,StrtConn,StrtConnquintile,ActCenDist,ActCenDistquintile,TobPopDensp1000,TobPopDensp1000quintile,TobTobVendDist,TobTobVendDistquintile,RatioTrailsToRoads,RatioTrailsToRoadsquintile
0,2,106,0,290,0.000146,1,0,0,0,0,1,1,0,1,0.000000,1,26140.126920,5,1711.468932,5,0.222163,4,0.000000,1,23445.162640,5,0.000000,1,24994.534480,5,0.007525,1,0.983319,1,32549.528700,5,0.000000,1,18751.682830,5,0.038605,1
1,22,7547,192,18953,0.386139,3,0,0,1,1,6,4,4,4,0.008632,2,1681.611727,2,217.118849,2,0.166304,4,0.103583,3,1141.854025,3,0.017264,2,1463.441728,3,5.802168,4,40.651591,4,6272.131139,2,0.480048,2,1019.618388,3,0.121901,2
2,41,1433,11,2621,0.450430,4,1,0,1,1,2,2,6,2,0.090396,5,1166.380004,2,138.261425,1,0.307801,5,0.361584,5,465.853680,1,0.000000,1,1608.507807,3,2.291587,3,34.391091,4,3946.259849,1,5.789271,5,433.933303,1,0.222852,4
3,0,271,5,690,0.000120,1,0,0,0,0,2,2,0,2,0.000000,1,21560.133640,5,2682.871535,5,0.004769,1,0.000000,1,16310.619260,5,0.000000,1,18311.678500,5,0.015401,1,0.567092,1,25884.481330,5,0.000000,1,14757.092960,5,0.000000,1
4,31,587,25,1578,0.140498,2,0,0,0,1,2,2,1,2,0.000000,1,6520.564030,5,923.261045,5,0.014819,1,0.000000,1,4379.501428,4,0.000000,1,4362.594630,4,0.066460,2,2.002509,1,11497.802890,4,0.000000,1,4362.594630,5,0.090617,2
5,43,736,20,2037,0.414610,4,0,0,0,1,2,2,2,2,0.000000,1,2412.145401,3,184.858326,1,0.088619,1,0.000000,1,2327.635718,4,0.000000,1,2306.150057,3,4.090075,4,29.478016,3,6702.915589,2,0.000000,1,2033.601867,4,0.046827,2
6,3,2226,49,3875,0.542890,5,0,0,1,1,6,4,5,4,0.000000,1,4927.240811,4,158.276079,1,0.554618,5,0.114735,4,717.333739,2,0.028684,2,1326.723488,2,3.978732,4,31.720207,3,23115.532760,5,1.413228,3,696.377342,2,0.292693,5
7,10,479,36,993,0.847546,5,1,1,1,1,7,5,7,5,0.037318,4,965.344308,1,229.975343,2,0.142600,4,0.621958,5,369.506218,1,0.024878,2,956.274668,2,7.481207,5,113.265985,5,1663.495404,1,9.479434,5,316.179185,1,0.172038,3
8,15,4275,0,1858,0.213630,2,1,1,1,1,6,4,6,4,0.035666,4,1371.957516,2,295.560507,3,0.024917,1,0.106997,3,991.774438,3,0.011889,2,1368.527751,2,0.428610,2,5.672784,2,20319.729240,4,1.617749,4,999.489893,3,0.138250,3
9,11,3412,66,7521,0.418636,4,0,0,1,1,2,2,2,2,0.000000,1,2334.840293,3,195.308128,1,0.211762,4,0.088528,3,734.854534,2,0.000000,1,2681.513753,4,6.138845,4,41.378732,4,21768.170070,4,0.419933,2,775.371825,3,0.063443,2


In [0]:
from google.colab import files

pre_processed_data.to_csv('pre_processed_data_with_labelencoding.csv') 
files.download('pre_processed_data_with_labelencoding.csv')